In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
import pandas as pd
import numpy as np
train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

In [3]:
print(train.shape)
print(test.shape)

(8693, 14)
(4277, 13)


In [4]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [5]:
train.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [6]:
test.isnull().sum()

PassengerId       0
HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Name             94
dtype: int64

In [7]:
train.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

In [8]:
train.nunique()

PassengerId     8693
HomePlanet         3
CryoSleep          2
Cabin           6560
Destination        3
Age               80
VIP                2
RoomService     1273
FoodCourt       1507
ShoppingMall    1115
Spa             1327
VRDeck          1306
Name            8473
Transported        2
dtype: int64

In [9]:
# drop those rows that have missing values for the target(Transported)
y = train['Transported'].astype('int64')
train.dropna(axis = 0, subset = ['Transported'], inplace = True)
train.drop(labels = ['Cabin'], axis = 1, inplace = True)
test.drop(labels = ['Cabin'], axis = 1, inplace = True)

test_passenger_id = test['PassengerId']

train.drop(labels = ['PassengerId'], axis = 1, inplace = True)
test.drop(labels = ['PassengerId'], axis = 1, inplace = True)

train.drop(labels = ['Transported'], axis = 1, inplace = True)


In [10]:
print(train.shape)
print(test.shape)
# test_passenger_id

(8693, 11)
(4277, 11)


In [11]:
train.drop(labels = ['Name'], axis = 1, inplace = True)
test.drop(labels = ['Name'], axis = 1, inplace = True)

In [12]:
catagorical_columns = [cols for cols in train.columns if train[cols].dtype == 'object']

numerical_columns = [cols for cols in train.columns if train[cols].dtype in ['float64', 'int64']]
catagorical_columns
numerical_columns

['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

In [13]:
def handle_missing(df):
    for columns in catagorical_columns:
        df[columns] = df[columns].fillna(df[columns].mode()[0])
    for columns in numerical_columns:
        df[columns] = df[columns].fillna(df[columns].mean())
    return df

In [14]:
new_train = handle_missing(train)
new_test = handle_missing(test)
# y_train = new_train.Transported
# new_train.drop(labels = ['Transported'], axis = 1, inplace = True)
new_train.head(19)
# new_train.isna().any()


,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.000000,0.0,0.000000
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.000000,549.0,44.000000
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.000000,6715.0,49.000000
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.000000,3329.0,193.000000
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.000000,565.0,2.000000
5,Earth,False,PSO J318.5-22,44.0,False,0.0,483.0,0.000000,291.0,0.000000
6,Earth,False,TRAPPIST-1e,26.0,False,42.0,1539.0,3.000000,0.0,0.000000
7,Earth,True,TRAPPIST-1e,28.0,False,0.0,0.0,0.000000,0.0,304.854791
8,Earth,False,TRAPPIST-1e,35.0,False,0.0,785.0,17.000000,216.0,0.000000
9,Europa,True,55 Cancri e,14.0,False,0.0,0.0,0.000000,0.0,0.000000


In [15]:
#one hot encode the catagorical variables in new_train and new_test
from sklearn.preprocessing import OrdinalEncoder

# s = (new_train.dtypes == 'objects')
# object_columns = list(s[s].index)

encoder = OrdinalEncoder()

label_X_train = new_train.copy()
label_X_test = new_test.copy()


label_X_train[catagorical_columns] = encoder.fit_transform(new_train[catagorical_columns])
label_X_test[catagorical_columns] = encoder.transform(new_test[catagorical_columns])

label_X_train.head(19)

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,1.0,0.0,2.0,39.0,0.0,0.0,0.0,0.000000,0.0,0.000000
1,0.0,0.0,2.0,24.0,0.0,109.0,9.0,25.000000,549.0,44.000000
2,1.0,0.0,2.0,58.0,1.0,43.0,3576.0,0.000000,6715.0,49.000000
3,1.0,0.0,2.0,33.0,0.0,0.0,1283.0,371.000000,3329.0,193.000000
4,0.0,0.0,2.0,16.0,0.0,303.0,70.0,151.000000,565.0,2.000000
5,0.0,0.0,1.0,44.0,0.0,0.0,483.0,0.000000,291.0,0.000000
6,0.0,0.0,2.0,26.0,0.0,42.0,1539.0,3.000000,0.0,0.000000
7,0.0,1.0,2.0,28.0,0.0,0.0,0.0,0.000000,0.0,304.854791
8,0.0,0.0,2.0,35.0,0.0,0.0,785.0,17.000000,216.0,0.000000
9,1.0,1.0,0.0,14.0,0.0,0.0,0.0,0.000000,0.0,0.000000


In [16]:
print(new_train.shape)
print(new_test.shape)

(8693, 10)
(4277, 10)


In [17]:
#train valid split for evaluation purposes
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(label_X_train, y, test_size = 0.2, random_state = 0)
val_X.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
3601,0.0,0.0,0.0,16.0,0.0,0.0,0.0,623.0,40.0,209.0
6057,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2797,0.0,0.0,2.0,31.0,0.0,517.0,1.0,301.0,60.0,4.0
7110,0.0,0.0,0.0,14.0,0.0,653.0,0.0,4.0,0.0,0.0
8579,1.0,1.0,0.0,30.0,1.0,0.0,0.0,0.0,0.0,0.0


In [18]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
# Define the model
my_model_1 = XGBClassifier(base_score=0.5, learning_rate=0.06, n_estimators = 100, max_leaves = 10)
# my_model_1 = CatBoostClassifier(iterations = 1500, learning_rate = 0.03, verbose = 0)
# Fit the model
my_model_1.fit(train_X, train_y)
preds = my_model_1.predict(val_X)
from sklearn.metrics import accuracy_score
print(accuracy_score(preds, val_y))

0.7924094307073031


In [19]:
final_model = XGBClassifier(base_score=0.5, learning_rate=0.06, n_estimators = 100)
final_model.fit(label_X_train, y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.06, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [20]:
predictions = final_model.predict(label_X_test)
len(predictions)
predictions.sum()/len(predictions)

0.5337853635725976

In [21]:
output = pd.DataFrame({'PassengerId': test_passenger_id,
                       'Transported': predictions.astype('bool')})
output.set_index('PassengerId')
output.to_csv('sample_submission.csv', index=False)
output

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True
